In [2]:
import tweepy
import csv
import json
import pandas as pd
from json import JSONEncoder
import datetime
import time
import ijson

In [3]:
consumer_key = "7PC3L2O8ywCONU4ZuJ5h757mv"
consumer_secret = "MMhJhIIolKU3ONdXpLC6oGzOwYGYwYgzMILc3etxZ0yjuNtdze"
access_key = "1115405592372187136-19lxDiitrBBDSdSaLdyqgK3fJzJxWT"
access_secret = "w2FsaXLITsGDy1PsRvqkdUdr78sKa6HYGcekcrB1p0xbM"
count = 0
file_object = None
file_name = None

class StreamListener(tweepy.StreamListener):


    def on_status(self, status):
        global file_object, count, file_name
        tweet = {}
        if not pd.isnull(status.user.location) and status.text.startswith("RT")==False:
            tweet['id']=status.id
            tweet['created_at']=status.created_at.isoformat()
            tweet['text']=status.text
            tweet['user']={'location':status.user.location}
            tweet['geo']=status.geo
            
            if file_object is None:
                file_name = int(datetime.datetime.now().timestamp() * 1e3)
                with open(f'out.json',"w",encoding="utf-8") as file_object:
                    json.dump({'results':[tweet]}, file_object, indent=4)
                count += 1
                print(tweet)
            elif count == 5000:
                file_object.close()
                count = 1
                file_name = int(datetime.datetime.now().timestamp() * 1e3)
                with open(f'out.json',"w", encoding="utf-8") as file_object:
                    json.dump({'results':[tweet]}, file_object, indent=4)
                print("New file created")
            
            else:
                with open(f'out.json',"r",encoding="utf-8") as file_object:
                    results = json.load(file_object)['results']
                    file_object.close()
                    
                results.append(tweet)

                with open(f'out.json',"w",encoding="utf-8") as file_object:
                    json.dump({'results':results}, file_object)
                    count += 1
            print(count)
    def on_error(self, status_code):
        print("Encountered streaming error (", status_code, ")")
        sys.exit()
    
    def on_timeout(self):
        print(sys.stderr, 'Timeout...')
        return True # Don't kill the stream
        print("Stream restarted")

if __name__ == "__main__":
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    streamListener = StreamListener()
    stream = tweepy.Stream(auth=api.auth, listener=streamListener,tweet_mode='extended')
    stream.filter(languages = ['en'] ,locations = [144.9,-37.8,145,-37.7])

{'id': 1390266215583518720, 'created_at': '2021-05-06T11:24:30', 'text': 'Just watched season finale Line of Duty. Just want to say, thanks so much for an amazing show, loved every single e… https://t.co/ydjLQoGPEZ', 'user': {'location': '647 Chapel Street, Melbourne'}, 'geo': None}
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


KeyboardInterrupt: 

In [4]:
from SentimentAnalysis import *

with open('out.json') as a:
    data = json.load(a)

# print(len(data['results']))
SentimentAnalysis_result = []

for i in range(len(data['results'])):
    result_dict = {}
    data_str = remove_features(fix_abbreviation(strip_non_ascii(data["results"][i]["text"])))
    polarity = sentiment_analysis(data_str)
    attitude = condition(polarity)
    result_dict['order'] = str(i)
    result_dict['polarity'] = float(str(polarity)[:5])
    result_dict['attitude'] = attitude
    SentimentAnalysis_result.append(result_dict)
    print('{0:<5}   {1:^15.3f}   {2:>10}'.format(i, polarity, attitude) + '\n')

filename_result = 'SentimentAnalysis_result.json'
with open(filename_result,'w') as file_obj:
    json.dump(SentimentAnalysis_result, file_obj)
SentimentAnalysis_result

0            0.326          positive

1            0.000           neutral

2            0.000           neutral

3            0.000           neutral

4            0.425          positive

5            0.375          positive

6            0.500          positive

7            0.000           neutral

8            0.140          positive

9            0.200          positive

10           0.000           neutral

11          -0.500          negative

12           0.000           neutral

13           0.000           neutral

14           0.000           neutral

15          -0.400          negative

16           0.377          positive



[{'order': '0', 'polarity': 0.325, 'attitude': 'positive'},
 {'order': '1', 'polarity': 0.0, 'attitude': 'neutral'},
 {'order': '2', 'polarity': 0.0, 'attitude': 'neutral'},
 {'order': '3', 'polarity': 0.0, 'attitude': 'neutral'},
 {'order': '4', 'polarity': 0.425, 'attitude': 'positive'},
 {'order': '5', 'polarity': 0.375, 'attitude': 'positive'},
 {'order': '6', 'polarity': 0.5, 'attitude': 'positive'},
 {'order': '7', 'polarity': 0.0, 'attitude': 'neutral'},
 {'order': '8', 'polarity': 0.14, 'attitude': 'positive'},
 {'order': '9', 'polarity': 0.2, 'attitude': 'positive'},
 {'order': '10', 'polarity': 0.0, 'attitude': 'neutral'},
 {'order': '11', 'polarity': -0.5, 'attitude': 'negative'},
 {'order': '12', 'polarity': 0.0, 'attitude': 'neutral'},
 {'order': '13', 'polarity': 0.0, 'attitude': 'neutral'},
 {'order': '14', 'polarity': 0.0, 'attitude': 'neutral'},
 {'order': '15', 'polarity': -0.4, 'attitude': 'negative'},
 {'order': '16', 'polarity': 0.377, 'attitude': 'positive'}]